# Setup

In [1]:
# %cd /content/drive/MyDrive/Rokomari Recommendation Dataset
%cd /content/drive/MyDrive/Dataset/rokomari_books

/content/drive/MyDrive/Dataset/rokomari_books


In [2]:
!ls

books.csv				 rokomari_bn_books_author_corrected.csv
books_modified_1.csv			 rokomari_bn_books.csv
preprocessing_bn_books_authors.ipynb	 rokomari_books_categories_corrected.csv
preprocessing_bn_books_categories.ipynb  rokomari_books_preprocessing.ipynb
preprocessing_bn_books_title.ipynb


In [3]:
# !pip install googletrans==4.0.0-rc1
!pip install googletrans==4.0.0rc1 --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00


In [4]:
!pip install langdetect --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
!pip install deep_translator --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00


# Import

In [6]:
import re
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import langdetect as ld
import googletrans as gt
import deep_translator as dt
from deep_translator import GoogleTranslator

# 1. Load Dataframe

In [7]:
df_bn = pd.read_csv("rokomari_books_categories_corrected.csv")

In [8]:
df_bn.head()

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed
0,লু স্যুন,পশ্চিমবঙ্গের বই: সমকালীন গল্প,9788194097303,No Ratings,No Reviews,320,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,নির্বাচিত গল্প সংকলন,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, সমকালীন গল্প"
1,আর্কাদি গাইদার,পশ্চিমবঙ্গের বই: শিশু-কিশোর উপন্যাস,9788193860939,No Ratings,No Reviews,180,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,তিমুর ও তার দলবল,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, শিশু-কিশোর উপন্যাস"
2,মতি নন্দী,পশ্চিমবঙ্গের বই: সমকালীন উপন্যাস,No ISBN,No Ratings,No Reviews,250,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,নায়কের প্রবেশ ও প্রস্থান,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, সমকালীন উপন্যাস"
3,উৎপলকুমার বসু,পশ্চিমবঙ্গের বই: বাংলা কবিতা,No ISBN,No Ratings,No Reviews,300,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,হাঁস চলার পথ,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, বাংলা কবিতা"
4,দিলরুবা হাসান,বয়স যখন ৪-৮: শিক্ষামূলক,No ISBN,No Ratings,No Reviews,129,No image,বৃত্তকলা একাডেমি,0.0,No summary,অনুভূতির ছোঁয়া,bn,No,bn,bn,"বয়স যখন ৪-৮, শিক্ষামূলক"


In [14]:
df_bn[df_bn.duplicated()]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed
3740,শ্রী বিমান বিহারী ঘোষ,পশ্চিমবঙ্গের বই,No ISBN,No Ratings,No Reviews,360,No image,বর্ণাশ্রম পাবলিকেশন হাউস,0.0,No summary,ভবঘুরের ভাবনা...,bn,No,bn,bn,পশ্চিমবঙ্গের বই
5199,বিনোদন বিচিত্রা,চলচ্চিত্র ও বিনোদন ম্যাগাজিন: মাসিক,No ISBN,No Ratings,No Reviews,25,No image,বিনোদন বিচিত্রা,0.0,No summary,বিনোদন বিচিত্রা - জুন ‘ ১২,bn,No,bn,bn,"চলচ্চিত্র, বিনোদন ম্যাগাজিন, মাসিক"
5930,গীতা সাহা,ভ্রমণ বিষয়ক স্মৃতি,No ISBN,No Ratings,No Reviews,180,No image,বোনা (ভারত),0.0,No summary,ভ্রমণে প্রতিবেশী রাষ্ট্র,bn,No,bn,bn,ভ্রমণ বিষয়ক স্মৃতি
9411,মনন কুমার মণ্ডল,পশ্চিমবঙ্গের বই: সাহিত্য সমালোচনা,No ISBN,No Ratings,No Reviews,540,No image,এবং মুশায়েরা (ভারত),0.0,No summary,আধুনিক বাংলা উপন্যাস ব্যষ্টি ও সমষ্টি,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, সাহিত্য সমালোচনা"
9896,পল্লব পত্রকার,পশ্চিমবঙ্গের বই,No ISBN,No Ratings,No Reviews,126,No image,একুশের ডাক (ভারত),0.0,No summary,উজাননাইয়া,bn,No,bn,bn,পশ্চিমবঙ্গের বই
15521,অনির্বান দে,পশ্চিমবঙ্গের বই,No ISBN,No Ratings,No Reviews,450,No image,খোয়াই পাবলিশিং হাউস (ভারত),0.0,No summary,স্বপ্নভূমি,bn,No,bn,bn,পশ্চিমবঙ্গের বই
15522,দেবসেনাপতি নন্দী,পশ্চিমবঙ্গের বই,No ISBN,No Ratings,No Reviews,450,No image,খোয়াই পাবলিশিং হাউস (ভারত),0.0,No summary,ইতি ভবানী পাঠক,bn,No,bn,bn,পশ্চিমবঙ্গের বই
15528,সুদীপ ভট্টাচার্য,পশ্চিমবঙ্গের বই,No ISBN,No Ratings,No Reviews,324,No image,খোয়াই পাবলিশিং হাউস (ভারত),0.0,No summary,রবি ঠাকুরের রান্না,bn,No,bn,bn,পশ্চিমবঙ্গের বই
16261,সুধীর চক্রবর্তী,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,No ISBN,No Ratings,No Reviews,1440,No image,লালমাটি প্রকাশন (ভারত),0.0,No summary,সুধীর চক্রবর্তী রচনাবলি -৩,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, রচনাসমগ্র, সংকলন"
21216,দীপক কুমার প্রামাণিক,পশ্চিমবঙ্গের বই,No ISBN,No Ratings,No Reviews,270,No image,নায়েক এন্টারপ্রাইজ (ভারত),0.0,No summary,অমৃতধারা 2য় খণ্ড,bn,No,bn,bn,পশ্চিমবঙ্গের বই
